In [1]:
import requests
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from googletrans import Translator
import time
import json
from together import Together

In [2]:
# Initialize the Together AI client, translator, and cache
client = Together(api_key="your-together-api-key")
cmc_api_key = "your-coinmarketcap-api-key"
cache = {}

# Define rate limiting
last_api_call_time = 0
rate_limit_interval = 5  # 10 seconds between API calls

In [3]:
# Tools
def fetch_crypto_price(crypto="bitcoin"):
    """Fetch cryptocurrency price. The price that you return should round up the price to 3 decimal places."""
    global last_api_call_time
    url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"

    header = headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': cmc_api_key,
    }

    parameters = {
      'start':'1',
#       'limit':'5',
      'convert':'USD'
    }

    # Check rate limit
    current_time = time.time()
    if current_time - last_api_call_time < rate_limit_interval:
        return f"Rate limit exceeded. Please wait {int(rate_limit_interval - (current_time - last_api_call_time))} seconds."

    # Check cache
    if crypto in cache and time.time() - cache[crypto]["timestamp"] < 60:  # Cache expires in 60 seconds
#         print("from cache", time.time() - cache[crypto]["timestamp"])
        return cache[crypto]["price"]

    try:
        response = requests.get(url, params=parameters, headers=headers).json()
        data = response['data']
#         print(data)
        for crypto_currency in data:
            if crypto_currency['name'].lower() == crypto.lower():
                price = crypto_currency["quote"]["USD"]["price"]
                cache[crypto] = {"price": price, "timestamp": time.time()}
                last_api_call_time = current_time
                return price

        return "Cryptocurrency not found. Kindly input the correct spelling for the crypto."
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        return f"Error fetching price: {e}"
    except KeyError:
        return "Unexpected response format."

In [4]:
def translate_text(text, target_lang="en"):
    """Translate text to target language (default: English)."""
    translator = Translator()
    try:
        detected_lang = translator.detect(text).lang
        if detected_lang != target_lang:
#             print("inside if because detected language is ", detected_lang)
            return translator.translate(text, src=detected_lang, dest=target_lang).text
        return text  # Already in the target language
    except Exception as e:
        return f"Error translating text: {e}"

In [5]:
# Function to interact with the Together AI LLaMA model
def llama_response(messages):
    try:
        response = client.chat.completions.create(
            model="meta-llama/Llama-3-8b-chat-hf",
            messages=messages,
        )
        
        result = response.choices[0].message.content
        return result
    
    except Exception as e:
        return f"Error interacting with LLaMA API: {e}"

In [6]:
# Define the tools dictionary
available_actions = {
    "fetch_crypto_price": fetch_crypto_price,
    "translate_text": translate_text,
}

# SYSTEM PROMPT
SYSTEM_PROMPT = """
You are an Intelligent AI Assistant.
You run in a loop of Thought, Action, Observation, Output.
At the end of the loop you output an Answer.
Wait for the user prompt.
Use Thought to understand the question you have been asked.
Use Action to run one of the tools available to you - then return observation.
Observation will be the result of running those tools.
Once you get the observation, return the AI response based on system prompt and tool_output.

Your main role is to assist users by fetching accurate and up-to-date information about Cryptocurrency prices and 
translating user queries into English if they are not in English and then respond to the query. 
Your responses must always be in English, regardless of the input language.

Here are your instructions:
1. If the user asks for the current Bitcoin price or any cryptocurrency price, use the "fetch_crypto_price" tool 
    to get the information. Wait for the tool's response before answering the user. 
    If the tool fails to provide the information, politely inform the user that the price could not be retrieved.
2. If the user's query is not in English, use the "translate_text" tool to detect and translate it into English. 
    Use the translated text as input for processing the user's request.
3. Maintain the conversation context, ensuring continuity across multiple queries. For example:
   - If the user asks for Bitcoin information and follows up with "What is it's price today?", infer that the second 
       query is also about bitcoin itself, user is asking about the price of bitcoin and update your thought as:
       { "role": "thought", "thought": "I should call the get_crypto_price function with 'bitcoin'." }
   - Avoid repeating tool calls unnecessarily if the context is already clear.
     If in the previous call you have already found the current price of bitcoin, then reuse that price and do not call the tool again.
   - Once you have an observation, make sense of it with context to earlier response and update your thought accordingly 
     to take the next action or to give the output.
4. Use tools only when necessary. For example:
   - If the user asks, any general questions about cryptocurrency, or any other general query, you can directly respond without calling any tool.
   - If the user query is in English and you understand it, no need to use the traslate_text tool.
5. If the input query or tool response is blank, and you think that you do not know the answer, respond with: "I’m sorry, I don’t have an answer to that."
6. For language-based queries, translate only the user input into English and return all responses in English.
7. Truncate the float values in the result upto 3 decimal places.

Strictly use the tools you are equipped with and do not generate a function of your own.

You are equipped with two tools:
- **fetch_crypto_price**: Use this to get the current prices of cryptocurrencies.
- **translate_text**: Use this to detect and translate user queries into English. Ensure your responses always remain in English.

Instruction about tools usage :
Only execute an action if the user's request explicitly mentions the need for data retrieval, external information, or content generation that cannot be directly answered with your internal knowledge.
Do not trigger the action utility unless the user's query contains clear actionable keywords like 'search', 'generate', 'fetch', or 'create'."

Example of when not to use any tool :
{ "role": "user", "user": "What is Bitcoin?" }
{ "role": "thought", "thought": "This is a general query, I do not need any tool." }
{ "role": "output", "output": "Bitcoin is a decentralized digital currency that allows for peer-to-peer transactions without 
    the need for intermediaries like banks. It was created in 2009 by an individual or group of individuals using the pseudonym 
    Satoshi Nakamoto." }
Output : Bitcoin is a decentralized digital currency that allows for peer-to-peer transactions without 
    the need for intermediaries like banks. It was created in 2009 by an individual or group of individuals using the pseudonym 
    Satoshi Nakamoto.

Example of how to use the fetch_crypto_price tool :
{ "role": "user", "user": "What is the price of Bitcoin?" }
{ "role": "thought", "thought": "I should call the fetch_crypto_price function with 'bitcoin'." }
{ "role": "action", "function": "fetch_crypto_price", "input": "bitcoin" }
{ "role": "user", "observation : 35000 " }
Once you have an observation from the tool call:
{ "role": "user", "tool_output" : "35000 "}
{ "role": "thought", "thought": "I have an observation for the bitcoin price from the tool call." },
{ "role": "output", "output": "The price of Bitcoin is 35000 USD." }
Output : The price of Bitcoin is 35000 USD.

Example of how you need to use the translate_text tool :
{ "role": "user", "user": "quelle est la capitale de l'Inde ?" }
{ "role": "thought", "thought": "I should call the translate_text function with 'quelle est la capitale de l'Inde ?'." }
{ "role": "action", "function": "translate_text", "input": "quelle est la capitale de l'Inde ?" }
{ "role": "user", "observation : What is the capital of India? " }
Once you have an observation from the tool call:
{ "role": "user", "tool_output" : "What is the capital of India? "}
{ "role": "thought", "thought": "I can answer this question directly now." }
{ "role": "output", "output": "The capital of India is New Delhi." }
Output : The capital of India is New Delhi.

"""

In [7]:
def ai_agent(user_input):
    messages = [{"role": "system", "content": SYSTEM_PROMPT},
               {"role": "user", "content": user_input}]

    while(1):
#         print ("----------AI Agent begin----------------")

        result = llama_response(messages)
        
        messages.append({"role": "assistant", "content": result})
        
#         print("Raw LLM Response:", result)  # Debugging step to see raw output
#         print("-----------response ends-------------------------")
        # Split the result by newlines to process each JSON block individually
        result_lines = result.split("\n")
        for line in result_lines:
            line = line.strip()  # Remove any leading/trailing whitespace
#             print("line in result lines ---------------", line)
            if not line:  # Skip empty lines
#                 print("skipping this line as it is empty", line)
                continue
        
            try:
#                     print("-----------line in try----------", line)
                    result_json = json.loads(line)  # Parse the current line
#                     print("Parsed JSON:", result_json)  # Debugging step to see parsed JSON
                    
                    # Handle roles based on parsed JSON
                    role = result_json.get("role", "")
#                     print("role", role)
                    
                    if role == "output":
                        # Return final output to the user
                        return result_json.get("output", "I'm sorry, I don't have an answer to that.")
                    
                    elif role == "action":
                        # Extract action details
                        function_name = result_json.get("function", "")
                        function_input = result_json.get("input", "")

                        if function_name == "fetch_crypto_price":
                            observation = fetch_crypto_price(function_input)
                        elif function_name == "translate_text":
                            observation = translate_text(function_input)
                        else:
                            observation = "Invalid function call."

                        # Add observation to the conversation context
                        messages.append({"role": "user", "content": f'observation : {observation}'})
#                         print("Observation:", observation)
                    
                        break  # Continue loop to process further lines
                    
                    
            except json.JSONDecodeError as e:
#                     print("JSON Parsing Error:", e)  # Debugging step to understand parsing issues
                continue  # Skip invalid lines and move to the next one

In [11]:
# Main loop to simulate conversation
if __name__ == "__main__":
    print("AI Agent: Hello! How can I assist you today?")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("AI Agent: Goodbye!")
            break
        response = ai_agent(user_input)
        print(f"AI Agent: {response}")

AI Agent: Hello! How can I assist you today?
You: What is Bitcoin?
AI Agent: Bitcoin is a decentralized digital currency that allows for peer-to-peer transactions without the need for intermediaries like banks. It was created in 2009 by an individual or group of individuals using the pseudonym Satoshi Nakamoto.
You: What is its current price?
AI Agent: The current price of Bitcoin is 102225.264.
You: How much money will be needed to buy 5 bitcoins?
AI Agent: The current price of Bitcoin is 102225.26482329183 USD. To buy 5 Bitcoins, you would need 5 x 102225.26482329183 = 510112.6224164586 USD.
You: What is indian rupees to usd exchange rate?
AI Agent: The Indian rupee (INR) to US dollar (USD) exchange rate is subject to fluctuations and can vary depending on various market and economic factors. As of now, the exchange rate is around 75.50 INR per USD.
You: If the exchange rate currently is 1 USD = 85 INR, then what is bitcoin's price in indian rupees?
AI Agent: The price of Bitcoin in 

In [15]:
# Main loop to simulate conversation
if __name__ == "__main__":
    print("AI Agent: Hello! How can I assist you today?")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("AI Agent: Goodbye!")
            break
        response = ai_agent(user_input)
        print(f"AI Agent: {response}")

AI Agent: Hello! How can I assist you today?
You: Hola
AI Agent: Hello! How can I assist you today?
You: ¿cuál es el precio de Bitcoin hoy?
AI Agent: The current price of Bitcoin is 102,446.151 USD.
You: ¿Y cuál es el precio de Ethereum?
AI Agent: The current price of Ethereum is 3162.595 USD.
You: Gracias por su ayuda hoy.
AI Agent: You're welcome! It was my pleasure to assist you today.
You: quit
AI Agent: Goodbye!
